<a href="https://colab.research.google.com/github/meltampsaol/TF2_0_NLP_tasks_with_Transformers/blob/main/chapter1_demo1_flask_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import output
import getpass

token = getpass.getpass("Enter your Hugging Face Token: ")
# Save it to an environment variable for use in the notebook
output.register_callback("huggingface_token", lambda: token)


Enter your Hugging Face Token: ··········


In [ ]:
from transformers import pipeline

generator = pipeline('text-generation', model='gpt2')
sentences = generator('Filipinos are known for')[0]['generated_text']
print('\n')
print(sentences)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




Filipinos are known for their beautiful women, and on the morning of the 9th, after sunrise they would gather in front of a big box and get the job done! The workers were always looking for workers! There are very few working day


In [ ]:
!pip install tensorflow-cpu --upgrade



In [ ]:
!pip install flash-attn

In [ ]:
!pip install flask transformers requests

In [ ]:
%%writefile flask_chatbot_app.py
from flask import Flask, request, jsonify
from transformers import pipeline
import tensorflow as tf
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

app = Flask(__name__)

# Load the Hugging Face model
chatbot_model = pipeline("text-generation", model="microsoft/Phi-3.5-mini-instruct", trust_remote_code=True, attn_implementation="eager", cache_dir='/content/transformers_cache')

@app.route('/chatbot', methods=['POST'])
def chatbot():
    user_input = request.json.get('message')
    if not user_input:
        return jsonify({'response': 'Please provide a message.'}), 400
    response = generate_response(user_input)
    return jsonify({'response': response})

def generate_response(user_input):
    try:
        # Generate a response using the Hugging Face model
        messages = [
            {"role": "user", "content": user_input},
        ]
        result = chatbot_model(messages, max_length=250, num_return_sequences=1)
        return result[0]['generated_text']
    except Exception as e:
        return str(e)

if __name__ == '__main__':
    app.run(debug=True, port=5000, host='0.0.0.0')


Writing flask_chatbot_app.py


In [ ]:

import subprocess
subprocess.run(['pkill', '-f', 'flask_chatbot_app.py'])

CompletedProcess(args=['pkill', '-f', 'flask_chatbot_app.py'], returncode=1)

In [ ]:
!nohup python flask_chatbot_app.py &

nohup: appending output to 'nohup.out'


In [ ]:
!sudo lsof -i -P -n | grep LISTEN

node         7 root   21u  IPv6  19903      0t0  TCP *:8080 (LISTEN)
kernel_ma   24 root    3u  IPv4  19672      0t0  TCP 172.28.0.12:6000 (LISTEN)
colab-fil   72 root    3u  IPv4  19911      0t0  TCP 127.0.0.1:3453 (LISTEN)
jupyter-n   93 root    7u  IPv4  20807      0t0  TCP 172.28.0.12:9000 (LISTEN)
python3    255 root   21u  IPv4  29888      0t0  TCP 127.0.0.1:39063 (LISTEN)
python3    306 root    3u  IPv4  30321      0t0  TCP 127.0.0.1:34497 (LISTEN)
python3    306 root    4u  IPv4  30322      0t0  TCP 127.0.0.1:48073 (LISTEN)


In [ ]:

import requests

# Define the URL of the Flask app
url = 'http://127.0.0.1:5000/chatbot'

# Send a request to the Flask app
response = requests.post(url, json={'message': 'Hello, how are you?'})
print(response.json())


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /chatbot (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f67355b2140>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
response = requests.post(url, json={'message': 'I was wondering how can I go to Eiffel Tower from the airport using the train and subway?'})
print(response.json())